In [1]:
import os
import requests
import json
from tqdm import tqdm
import re
import pandas as pd
from pandas import json_normalize
import time
from dotenv import load_dotenv
load_dotenv()
import openai
# from openai import OpenAI
import whisper
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.schema import Document
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.mapreduce import MapReduceChain
from langchain.chains import (
                StuffDocumentsChain,
                LLMChain,
                ReduceDocumentsChain,
                MapReduceDocumentsChain,
            )
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.openai_functions import (
    create_structured_output_runnable,
)
from pydantic import BaseModel, Field
from typing import List, Optional
from typing import Sequence
from langchain.pydantic_v1 import BaseModel, Field
from langchain.document_loaders import TextLoader
from IPython.display import Markdown, display
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [2]:
import backoff
from tqdm import tqdm

@backoff.on_exception(backoff.expo, (requests.exceptions.RequestException, requests.exceptions.HTTPError), max_tries=8)
def fetch_page(url):
    response = requests.get(url)
    response.raise_for_status()  # Raises a HTTPError if the status is 4xx, 5xx
    return response.json()

def fetch_audio_data(base_url, start_page, end_page):
    all_results = []
    for page in tqdm(range(start_page, end_page + 1)):
        url = f"{base_url}?page={page}"
        data = fetch_page(url)
        all_results.extend(data["results"])
        time.sleep(1)
    df = pd.DataFrame.from_records(all_results)
    return df


In [3]:
# Define the URL for the audio endpoint
base_url = "https://www.courtlistener.com/api/rest/v3/audio/"

# Call the function to fetch the data
df = fetch_audio_data(base_url, 1, 10)

100%|██████████| 10/10 [00:15<00:00,  1.54s/it]


In [4]:
df.shape

(200, 24)

In [5]:
df.head()

,resource_uri,id,absolute_url,panel,docket,date_created,date_modified,source,case_name_short,case_name,case_name_full,judges,sha1,download_url,local_path_mp3,local_path_original_file,filepath_ia,ia_upload_failure_count,duration,processing_complete,date_blocked,blocked,stt_status,stt_google_response
0,https://www.courtlistener.com/api/rest/v3/audi...,89939,/audio/89939/masters-v-illinois-workers-compen...,[],https://www.courtlistener.com/api/rest/v3/dock...,2023-12-20T14:31:48.524985-08:00,2023-12-21T01:26:55.575546-08:00,C,Masters,Masters v. Illinois Workers' Compensation Comm'n,,,aef1e535d8af2ec5412662f42ef5fb697c70b0d6,https://ilcourtsaudio.blob.core.windows.net/an...,mp3/2023/12/20/masters_v._illinois_workers_com...,mp3/2023/12/20/masters_v._illinois_workers_com...,https://archive.org/download/gov.uscourts.illa...,None,1801.0,True,None,False,0,
1,https://www.courtlistener.com/api/rest/v3/audi...,89938,/audio/89938/osman-v-illinois-workers-compensa...,[],https://www.courtlistener.com/api/rest/v3/dock...,2023-12-20T14:31:43.921858-08:00,2023-12-21T01:26:30.037146-08:00,C,Osman,Osman v. Illinois Workers' Compensation Comm'n,,,5008a589d4daaed1b80c849cd5f063d4976d6eae,https://ilcourtsaudio.blob.core.windows.net/an...,mp3/2023/12/20/osman_v._illinois_workers_compe...,mp3/2023/12/20/osman_v._illinois_workers_compe...,https://archive.org/download/gov.uscourts.illa...,None,1884.0,True,None,False,0,
2,https://www.courtlistener.com/api/rest/v3/audi...,89937,/audio/89937/smithfield-foods-inc-v-illinois-w...,[],https://www.courtlistener.com/api/rest/v3/dock...,2023-12-20T14:31:36.315398-08:00,2023-12-21T01:26:29.022593-08:00,C,,"Smithfield Foods, Inc. v. Illinois Workers' Co...",,,3e1f0e82a9cabd1847749f6e5a2fed6068f8d2df,https://ilcourtsaudio.blob.core.windows.net/an...,mp3/2023/12/20/smithfield_foods_inc._v._illino...,mp3/2023/12/20/smithfield_foods_inc._v._illino...,https://archive.org/download/gov.uscourts.illa...,None,1606.0,True,None,False,0,
3,https://www.courtlistener.com/api/rest/v3/audi...,89936,/audio/89936/walsh-v-illinois-workers-compensa...,[],https://www.courtlistener.com/api/rest/v3/dock...,2023-12-20T10:31:47.005867-08:00,2023-12-21T01:26:32.074850-08:00,C,Walsh,Walsh v. Illinois Workers' Compensation Comm'n,,,1665f4d14a2b47578fcfc007e0d1ebb4f6575f73,https://ilcourtsaudio.blob.core.windows.net/an...,mp3/2023/12/19/walsh_v._illinois_workers_compe...,mp3/2023/12/19/walsh_v._illinois_workers_compe...,https://archive.org/download/gov.uscourts.illa...,None,1354.0,True,None,False,0,
4,https://www.courtlistener.com/api/rest/v3/audi...,89935,/audio/89935/cronk-v-illinois-workers-compensa...,[],https://www.courtlistener.com/api/rest/v3/dock...,2023-12-20T10:31:43.251363-08:00,2023-12-21T01:26:27.937690-08:00,C,Cronk,Cronk v. Illinois Workers' Compensation Comm'n,,,0273d05d71117d096d4c2274a50bb195a8e793fb,https://ilcourtsaudio.blob.core.windows.net/an...,mp3/2023/12/20/cronk_v._illinois_workers_compe...,mp3/2023/12/20/cronk_v._illinois_workers_compe...,https://archive.org/download/gov.uscourts.illa...,None,2244.0,True,None,False,0,


In [6]:
df['docket'].head().tolist()

['https://www.courtlistener.com/api/rest/v3/dockets/68106129/',
 'https://www.courtlistener.com/api/rest/v3/dockets/68106126/',
 'https://www.courtlistener.com/api/rest/v3/dockets/68106123/',
 'https://www.courtlistener.com/api/rest/v3/dockets/68104486/',
 'https://www.courtlistener.com/api/rest/v3/dockets/68104485/']

In [7]:
def search_insurance(df, column):
    df['insurance_found'] = df[column].apply(lambda x: 1 if 'insurance' in x.lower() else 0)
    return df

df = search_insurance(df, 'case_name')

In [8]:
print(df[df['insurance_found']==1].shape)
df[df['insurance_found']==1]['case_name'].tolist()

(4, 25)


['Jarrett Cudd v. State Farm Mutual Automobile Insurance Company',
 'Madelaine Chocolate Novelties, Inc. v. Great Northern Insurance Company',
 'Cassidy v. New York State Insurance Fund',
 'James Snell v. United Specialty Insurance Company']

In [9]:
def add_rows_to_excel(input_df, excel_file):
    """
    Adds rows to an excel file. If the excel file doesn't exist, creates it. 
    If it does exist, only adds the new rows from the input DataFrame. 
    If the data overlaps, only the distinct rows are saved in the final excel file.

    Args:
    input_df (pd.DataFrame): The DataFrame to add to the excel file.
    excel_file (str): The path to the excel file.
    """
    # Convert lists to strings in input_df
    input_df = input_df.applymap(lambda x: str(x) if isinstance(x, list) else x)

    if os.path.exists(excel_file):
        # Load the existing data
        existing_df = pd.read_excel(excel_file)

        # Convert lists to strings in existing_df
        existing_df = existing_df.applymap(lambda x: str(x) if isinstance(x, list) else x)

        # Concatenate the existing data with the new data
        combined_df = pd.concat([existing_df, input_df])

        # Remove duplicates
        final_df = combined_df.drop_duplicates()

        # Print the total number of rows and how many new rows were added
        print(f"Total number of rows: {len(final_df)}")
        print(f"Number of new rows added: {len(final_df) - len(existing_df)}")
    else:
        final_df = input_df
        # Print the total number of rows as the excel file didn't exist before
        print(f"Total number of rows: {len(final_df)}")
        print(f"Number of new rows added: {len(final_df)}")

    # Write the data to the excel file
    final_df.to_excel(excel_file, index=False)

In [10]:
add_rows_to_excel(df, "court_listener_oral_argument_listing.xlsx")

Total number of rows: 200
Number of new rows added: 200


In [11]:
insurance_df = df[df["insurance_found"] != 0]

i = 0

name = insurance_df.iloc[i]["case_name"]
url = insurance_df.iloc[i]["download_url"]
print(name)
print(url)

Jarrett Cudd v. State Farm Mutual Automobile Insurance Company
https://www.ca11.uscourts.gov/system/files_force/oral_argument_recordings/22-13916.mp3?download=1


In [12]:
def download_mp3(url, file_path):
    """
    Downloads an MP3 file from the given URL and saves it to the specified file path.

    Args:
    url (str): The URL of the MP3 file.
    file_path (str): The local path where the file should be saved.
    """
    try:
        # Send a GET request to the URL
        response = requests.get(url, stream=True)

        # Raise an exception if the request was unsuccessful
        response.raise_for_status()

        # Open a file in binary write mode
        with open(file_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192): 
                # Write the content chunk by chunk to the file
                file.write(chunk)

        print(f"File downloaded and saved as {file_path}")
    except requests.RequestException as e:
        print(f"An error occurred: {e}")

In [13]:
download_mp3(url, f"./data/audio/{name}.mp3")

File downloaded and saved as ./data/audio/Jarrett Cudd v. State Farm Mutual Automobile Insurance Company.mp3


In [14]:
%%time
model = whisper.load_model("base")
result = model.transcribe(f"./data/audio/{name}.mp3")
Markdown(f"{result['text']}")

CPU times: total: 21min 39s
Wall time: 2min 47s


 I will accept the court's invitation to jump directly to the heart of my argument, and it is this. The district court held that providing state farm with a meaningful opportunity to invoke the imprasal process before filing suit is a condition precedent to filing this action. That was error because the policy language simply doesn't say that. The appraisal provision says either party may request an appraisal. It doesn't say the policy holder has to request an appraisal before filing suit. It doesn't say the policy holder has to give state farms some kind of notice and opportunity to request appraisal before filing suit. That language about meaningful opportunity and so forth is entirely in the district court's opinion and completely absent from the policy. Well, let's say we agree with you in terms of that not being a precondition to suit, but once the written request for an appraisal was received by your client, isn't that a provision had to be complied with? Not as a condition for filing or maintaining suit. Right. We're going to give you that, but in terms of once the letter or request will receive. So state farm did move to compel an appraisal. We did oppose it. The district court didn't resolve that. We have arguments about why appraisal is not required for this particular claim under the Georgia Supreme Court decision in McGowan. And due to the nature of the systemic automatic and bad faith discount being applied by state farm. But in any case, I mean, wouldn't it be, I mean, even setting aside all of those things just for the moment, wouldn't it be important for damages purposes? If the district court compels us to engage in appraisal after you send us back down as I certainly hope you will, we would of course comply with that. And then turning back to the other part of it. I mean, the contract does, at least it seems to me, does require you once it's triggered, once the provision is triggered, which it is by request by state farm to engage in the appraisal process. And I'm going to add the same caveat just to make sure we're all on the same page, not as a condition of filing suit or maintain. I agree. Not as a. And so, so the answer to that. I look at McGowan, even before the Supreme Court or the appeal, the trial court, it ordered the enforcement of the appraisal provision, right? Certainly did. So the propriety of that order, compelling arbitration, wasn't an issue in McGowan. What was an issue in McGowan is whether that appraisal after completed and paid by state farm, somehow mooted or negated or resolved. And so, on other things, the very breach of contract claim that we raise here. I would add this to, I'm not sure where you're thinking of going with this, but I don't think this court could enter an order saying, we are reversing because this is not a condition precedent, but district court or, you know, Frank's client, you have to engage in appraisal after the case goes back down. And the reason I don't think you could do that is that wouldn't be a right for any reason. You'd be reversing what the district court did and state farm filed no appeal, sorry, no cross appeal conditional or otherwise, asking you for that relief. And they would have had to, jurisdictionally, if you're going to do something other than affirm what the district court did. I'm sorry. So, let's say we agree with you that the appraisal wasn't necessary as a precondition to suit, but that we will interpret and then therefore enforce the provision with respect to once a written request for an appraisal is received, that must be satisfied. So, then what would the relief be in that instance? We agree with you on one side, but we believe your client hasn't fully satisfied other obligations. You could do half of that. The only half of that you can do is to tell the district court, you know, you've got to reinstate the suit. I'm just going to give council a second. It's okay. No, that's okay. Now, you can tell the district court that you can reverse the district court. You must reverse the district court and reinstate the lawsuit. What happens after that within appraisal is not before you. The district court didn't reach that question. So, instead of just so your argument is that instead of dismissing this without prejudice that the district court should have stayed the action for the appraisal process to be to occur. I don't agree that the district court should have stayed the action. Certainly don't agree with that. I for the reason stated the district court also don't believe that an appraisal is required. But our business today is simply whether it was a condition precedent, because for you to reach an issue, the district court hasn't and said, you know, here's how you have to rule in the future, you have to compel an appraisal would be beyond the scope of what you do here. But in constring the provision and concluding, and I only speak for myself and just a question at this point is if we concluded that it was not a condition precedent. The way, at least I think you would get there, is by applying, of course, a plain language. And saying that the reason it's not a condition precedent is because what triggers it is a request from state farm. And the request from state farm did not, there's nothing about, there's nothing else in the language of the contract that requires you to run an appraisal. Before you file a lawsuit, the only thing that triggers the appraisal is a request from state farm. And so that kind of does beg the question about, okay, now state farm has has made the request for an appraisal. So that is something that the district court would have to address if we were to remand on that basis. And then it seems to me that there might be at least three options the district court would have. It could agree that it could decide that the case should be dismissed without prejudice while the appraisal process goes forward. It could abate the case while the appraisal process goes forward or it could conclude that the case could go forward while the appraisal process goes forward. But in any case, and in all of those cases, I think consistent with a plain language of your contract. At least under the hypothetical I'm proposing the district court would have to conclude that the appraisal would go forward. So why is that not so? I don't agree with the first of the three options you posed, which was dismissing the case without prejudice for an appraisal because that's simply another way of treating the appraisal provision as a condition precedent. The other two options, which I think were to stay pending appraisal or I suppose it's not an appraisal. I guess having appraisal but not a stay. Those are things the district court could do. I don't know whether the district court has to do them. We have arguments about why the district court does not. But again, that's not what we're here on. Thank you. I just want to clarify with respect to Judge Rosenbaum's first option, which you now rejected, is that essentially a dismissal is equivalent or let's say requiring that the appraisal happen, which would essentially state the litigation is the equivalent of essentially making the appraisal a precondition to suit. No, if I understood Judge Rosenbaum correctly, the first hypothetical or the first possibility was the district court would dismiss without prejudice pending an appraisal. I mean, that's what the district court's already done. That is treating the appraisal as a condition precedent. Okay, I think the same thing. Maybe I think so too, Your Honor. I think so too. I know time is running. I want to briefly address a provision. I think my brother will bring to your attention that says legal action may not be brought against us unless there has been full compliance with the policy. Our point on that is very simple. At the time we brought suit, no one had requested an appraisal. We were in full compliance with the policy. That argument simply begs the question of whether appraisal is a condition precedent. Okay, we acknowledge an unpublished decision more versus travelers by this court. It is of course not binding. It is not persuasive with all respect. It was decided under Alabama law. Alabama law. Alabama may trump Georgia and football, but Georgia controls controls role tied your Honor. Georgia law controls this dispute. I'm secretly supporting. So time's winding down and we've descended into football. Perhaps I should sit down and maybe it's time. Thank you. Your. Mr. Diffie. Good morning. Your Honours may please the court. Dan Diffie on behalf of State Farm. I apologize for my brief coughing fit back there. It was an old lawyer's trick to distract but it was not. The solution to the problem that we've been talking about has passed. The solution was for the plaintiff to agree to the appraisal that was requested. They didn't do that. The policy is as clear as day that once it's requested, I'll get to condition proceed in just a minute. But once either party requested, it needs to go forward. The Georgia courts routinely enforce these appraisal provisions in policies like this. Instead, the plaintiff's. Sorry to interrupt. Let's say we agree with you that once they requested, and I don't know I haven't spoken with my colleagues about this, but once let's say that we agree that once they requested, the appraisal has to go forward. That doesn't answer what happens to the rest of case while that's happening. Correct. So I'll get to that. I'll get to that now, Judge. I'd like to respond by answering it this way. Imagine if instead of filing the suit first, plaintiff sent a letter to State Farm and said like he does in the complaint, you owe me an additional $670.25 for the total loss value of my car. I disagree with how you did it. And State Farm and response said, okay, we disagree with you. We're going to demand the appraisal. And then the parties, again, pre-suit, go down that path and the appraisal process is going forward. While that's happening, if the plaintiff marched into federal court and said I'm filing a lawsuit for breach and you didn't pay me or I don't like my calculation of total loss value, well, the court would dismiss that case in a minute, right? It would say you're going through this appraisal process, you're following the terms of the policy dismissal is appropriate. We're really no different here because what the plaintiff did was just go ahead and for the first time they noticed, they notified State Farm that they disagreed with the calculation, they happened to do it through a lawsuit. It doesn't change the requirements of the policy. It doesn't change the analysis of whether or not a claim should go forward in federal court. And that's what Judge Land in the District Court got exactly correct when he said it was premature. It's no different than if we were in the world of pre-suit. We're in suit now. They filed the lawsuit. State Farm requested the appraisal. The appraisal process is got to go forward. Dismissal appropriate and why is dismissal appropriate because we don't know what's going to happen through that appraisal process. But it doesn't really matter what happens through that appraisal process based on the allegations and the complaint because the allegations and the complaint are that. State Farm systematically and always applies this eight to nine percent. I guess. You know, cut to to whatever the price otherwise would be. And and it's basically a fraudulent kind of operation. And so if you want to challenge that, then you've got to you're always going to have to pay these attorneys fees or appraisal fees or et cetera, which would be fine. If it weren't always going to be the case uniformly in every single situation where replacement value was provided that there was going to be this eight to nine percent cut. And that's what the argument is right and so. So there are certain damages that would natural flow from that situation if it turned out that situation was true. And I wonder what your response to that your honor. I disagree if you as to the underlying premise that the appraisal is almost a necessary step, but you didn't like my idea about why dismissal was appropriate because when I said we don't know what's going to happen the appraisal. What if during the appraisal process the valuation comes back lower than what state farm provided. What if the parties have an issue with how the appraisal process was conducted. What if the state farm insured says wow, I got a little bit more money. That's great. I don't want to continue to proceed or litigate or anything like that. Any number of things where could happen, which is why such land was correct ruling that the case to premature. That shouldn't be presupposing what's going to happen. Well, we already know for sure that one of the things that will happen is there will be a payment for praisels for praisers, right? I mean, we know that is going to happen. And the problem here is that that is baked in through every single replacement cost that is offered. So you're always going to have to pay these appraisal fees or these appraisal fees again. Regardless of whether or not, you know, it was justified in the case to apply the so-called negotiation fee. And the question is the meaning of cash value in the contract. And so that is why it listed seems to me that there might be a reason for continuing the case in the meantime while the appraisal is going on. Your Honor, the actual cash value, let me take the fact as to the appraisal process and the fees that are incurred by the parties as part of that are part of the contract. So they are, they are, that's part of the policy. It's no different than having to pay a premium or anything like that. It is just part of the agreement between state farm and its insures. Whether or not those become, if your question is recoverable or part of the complaint or added to the case, well, that's all stuff that could happen much later. We don't know what's going to happen through the appraisal process. Everything we're talking about other than the fact that the parties are going to incur fees and doing it, pursuant to the terms of the contract, we don't know what's going to happen. And that's why dismissal is appropriate. That's the problem. I agree with you that in the normal, in the normal course of, you know, considering the contract that appraisal fees would not be an issue at all. The reason that they are allegedly an issue here is because the plaintiff is alleging that in every single case, you don't calculate a fair cash value. That your definition of cash value, which was not disclosed previously, is a fraudulent representation of cash value because it gives a haircut of 8% to 9% on the actual cash value in every single case. And so it's impossible to get what is actual cash value in any case before and unless you pay the appraisal fees, and if you do that, then you're going to have to necessarily incur these appraisal fees. And so that makes your actual cash value, even if it's 100%, less than your actual cash value should be because in every single case, you must do this. This is a cost of applying with a requirement in the contract in every single case. That's the problem that they're alludging. You're honored, but with the appraisal provision, the appraisal provision and the associated cost that come with it, don't vary or are not triggered by a particular insurance theory or disagreement. So one of the insured things, the actual cash value that State Farm presented through its processes was by fraud, by I just think my car is worth more money than State Farm does because the underlying comps were wrong. That's all irrelevant for the purposes of the appraisal provision. And again, coming back to where do we go if the appraisal provision is appropriate. Mr. Cud could go through the process and say, you just use the wrong comps or gosh, like we all do, we all think our stuff's worth more, right? What if the haircut was 90% if the haircut in every single case was 90% of what otherwise would have been the actual cash value? Would you agree that in that situation, the appraisal cost would be part of the damages that the person would experience in the allegation that this was a fraudulent scheme to undercut how much was actually due to every single one of your clients? You're on the other side of the stream. I appreciate that the court was asking that. I do want to say that I realize we're not here to dive into the weeds of the negotiation adjustment processes, but it's not a haircut. But even in your situation, your honor. Frankly, if it got to a point where it was just 90% and people with working knowledge of a particular car is worth $10,000 in state farm is offering $1,000, where first of all, state farm wouldn't do that. Second, there is a mechanism, right? And frankly, I would think if it was at that extreme, you'd get a letter back immediately or an email from the insurance saying, well, wait a second, this is so far off. I'm just a process here, and I realize we're at the motion to dismiss days, but there is an ongoing dialogue. Can I get any answer to the question, though? I mean, in that situation, would you agree that there would be a claim that could go forward right away? Not until the appraisal process was finished. Even though in every single case, there's a 90% haircut. You're on the, I think it's so unlikely we would get there, but the appraisal provision, again, is not triggered by why? So I think that- Hypocetical? No, I agree. But the appraisal provision is not only triggered if somebody has a particular theory. It applies no matter what. All right. So let me ask this. You said your opponent rejected participating in the appraisal process. The district court, of course, dismissed the suit and interpreted the appraisal provision as a condition preceding. Your adversary has challenged that as an era of law and said, if participation in the appraisal process is required, it's ready to do so. Why can't we vacate and remand? Because it was legal error to conclude that participation in the appraisal process was a condition preceded. We say not a condition preceded. And just send it back. You're on our think you could do that. But the proper course would be- Why would it be legal error to do that? Why would it be legal error to- To do it that way. To look at this, I look. We don't think the appraisal provision given that the request for appraisal didn't happen until after the suit was filed. We don't think that that can be viewed as a condition preceded. So it's just the district court erred and determining otherwise. And so we vacate the dismissal without prejudice based on that legal error sent back to the district court. That leaves everybody back in the position where the district court can say, okay. I think you have to participate, Mr. Cudden and appraisal, but upper shut up. You're on the- Because it would be legal error because there- You can also judge- What Judge Land ruled was that the case was premature. And I realize it's somewhat circular. That can't be right. Well, but it can't be. I really- Under my hypothetically can't. Right. Right. If we say it's not a condition preceded, that has to be wrong, doesn't it? Not in this- Not in this factual pattern, Your Honor. I think it's because I take it, you know, condition preceding when you think about that, right? Usually means there's some sort of performance that's triggered. Well, I don't know how it could be fine if- I'm pretty mature. If there was no request for an appraisal before a suit was filed. Because the request, excuse me, the filing of the suit was the insured's first time. They did it through counsel, filing a lawsuit in federal court, letting State Farm know that they disagreed. I go back to what I said earlier. If this came through a letter beforehand, again, that case would be- It would be dismissed in the minute. This circumstance is no different. Whether you call it just premature, whether you look at it, like we do in the district court, understanding analysis. Whether you look at it as a condition precedent, the- The State Farm did not have any notice of a disagreement. So no one knew that appraisal was going to be an issue until they filed suit. They filed suit. Appraisals required. Well, then it's appropriate to- The appraisal was required by the filing of the suit. Appraisal was required because after the suit was filed, State Farm required- It requested it. Isn't that right? Yes, and I agree. I misspoke their judge. So I don't see how we could be conditioned preceded. You don't see how it could be- You could say it's premature. It is premature. I mean, it could be that State Farm wouldn't request an appraisal, right? Exactly right. And then wouldn't be here, but once State Farm- Which is- Which means it wasn't much premature. No, you're honor. But if- I realize it's circular. But- But- There's no- Okay, let me just ask you, your client wrote this contract, right? Yes. And so- If you wanted to have- If you wanted to have sort of an exhaustion requirement before a lawsuit was filed, you could have written it that way, couldn't you? I do. Could- Could- Could- Could- Could- Could- Could- Could- Could- Could- Could- Could- Could- Could- Could- Could- Could- Could- Could- Could- Could- Could- Could- Could- Could- Should- Should- Should- Should- Should- And then we're going to be done. Okay. All right. So, just as it relates to the issue of it being premature, the- value until they file suit. Before that, the plaintiff had the opportunity to request a praisle. After that, the plaintiff had the opportunity to request a praisle and then, of course, state farm did. And state farm did. As soon as they did that, their lawsuit became premature. I realize it came after, but this just allows people to try and have these types of arguments about McGowan and all that. I feel like we've moved on from that, from the briefs, but that's where I am on the on the premature issue. Judge Boone. My question has to do with the fact that Mr. Cut filed this action as a class action. And so what impact, if any, does your position that once the written praisle is received, it halts, it should halt the litigation, what impact does that have in terms of being able to proceed on the underlying claim that's being raised as Judge Rosenbaum articulated the purposes of advancing a class. So that's part of it. And then the second thing, in the class definition, I don't believe right now, it includes a requirement that every member of the class have gotten an appraisal. So does that mean that the class definition from your perspective also would need to be amended? So two things on that first on the initial question. At this point in time, the the trial court is only going to test the claims of the name plaintiff as alleged in the. There's not been a class certified as it relates to that, Judge Boone, who the court probably picked up from the briefs. This is one of many of these cases pending around the country, different different plaintiffs, different states, slightly different policies from state to state about these total loss evaluations. All of those issues about what happens to class certification, the class definition appraisal for each class member. Your Honor, I think that's great foresight for what's going to be coming next in these cases, but respectfully we're not there yet in this particular case just trying to deal with the threshold. I don't think that was before the threshold question, the appraisal. Right, but let's say he goes ahead and get the appraisal for whatever reason, he's still not satisfied with the outcome. He moves forward with this case with a proposed putative class. Does he need to therefore amend the class definition to include the appraisal requirement? I think you would have to. One, yes, Your Honor, it would also need to amend this complaint to address all of those facts. And then of course, we would deal with the other insurers at class certification and applying the appraisal requirement to those insurers as well down the road. Okay, thank you very much, Your Honor. Thank you, Mr. Difflee. Mr. Larry, you saved two minutes. Oh, I'm sorry. I'm sorry. But I don't plan to use five unless you all have a substantial number of questions for me. I had a few thoughts on what my brother said before I do do you want to lead off with any questions? Any of you? I'm curious your thoughts on the class action question. I understand that maybe we're getting too far ahead. Sure. But I do see that as a natural consequence in terms of how these cases come forward. So to preview, there's a substantial base of Georgia authority holding that conditions like this can be satisfied by a named plenith on behalf of the entire class. It's also in federal law, for example, the filing of a single charge of discrimination by a Title VII plenith can exhaust that administrative requirement for the entire class. I'm confident we're going to win that argument when they raise it. The premature argument is just another way of saying condition precedent. There's no daylight there and no difference there. That's my first thought. My second thought is appraisal, the appraisal requirement can't possibly be a basis to dismiss this lawsuit because that's really the issue in McGowan. I mean, as Judge Rosenbaum articulated under McGowan, there is a breach of contract claim here that must be resolved in court. The last footnote in McGowan, by the way, refutes a colleague's argument that the appraisal fees are not damages because they're required by the policy. Speaks directly to and rejects that argument, adopting essentially the same analysis you just gave Judge Rosenbaum. This may be bombing the rubble, but as far as the idea that a post-suit refusal could be some basis to dismiss, the only condition precedent in the policy talks about you have to have been in full compliance when the action is brought. Federal precedent and state precedent, and this is discussed in our Plybri 21 to 23, is very well established that brought means filed. It doesn't mean maintained or continued. If you thought it was even a close issue or a question, you'd have to resolve it for us because we're the insured and they wrote the policy. Those were my thoughts on our bottle. Are there any questions I can answer? Thank you, Mr. Lowry. Thank you, sir. We are always appreciate Tom being given back to us. We're going to be in recess.

In [15]:
def save_dict_to_txt_file(data_dict, file_name):
    """
    Saves the 'text' value from a dictionary to a .txt file.

    Args:
    data_dict (dict): The dictionary containing the data.
    file_name (str): The name of the .txt file to save the data to.
    """
    # Get the 'text' value from the dictionary
    text = data_dict.get('text', '')

    # Clean the file name by removing spaces, punctuation, and special characters
    file_name = re.sub(r'\W+', '', file_name)

    # Create 'transcriptions' directory if it doesn't exist
    if not os.path.exists('data/transcriptions'):
        os.makedirs('data/transcriptions')

    # Open the file in write mode
    with open(f"data/transcriptions/{file_name}.txt", 'w', encoding='utf-8') as file:
        # Write the text to the file
        file.write(text)

    print(f"Data saved to data/transcriptions/{file_name}.txt")


In [5]:
def read_text_from_txt_file(file_name):
    """
    Reads the text from a .txt file.

    Args:
    file_name (str): The name of the .txt file to read the data from.
    
    Returns:
    str: The text read from the file.
    """
    # Check if the file exists
    if not os.path.exists(f"data/transcriptions/{file_name}.txt"):
        print(f"File data/transcriptions/{file_name}.txt does not exist.")
        return None

    # Open the file in read mode
    with open(f"data/transcriptions/{file_name}.txt", 'r', encoding='utf-8') as file:
        # Read the text from the file
        text = file.read()

    return text

In [17]:
save_dict_to_txt_file(result, name)

Data saved to data/transcriptions/JarrettCuddvStateFarmMutualAutomobileInsuranceCompany.txt


In [6]:
text = read_text_from_txt_file("MannievIllinoisDepartmentofInsurance")

In [19]:
import tiktoken

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [20]:
num_tokens_from_string(text, "cl100k_base")

3573

In [9]:
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=15000,chunk_overlap=0,separator="\n")

def run_map_reduce(input_doc, MAP_PROMPT, REDUCE_PROMPT):

    llm = ChatOpenAI(model_name="gpt-4-1106-preview", temperature=0.1)
    map_llm_chain = LLMChain(llm=llm, prompt=MAP_PROMPT)

    llm = ChatOpenAI(model_name="gpt-4-1106-preview", temperature=0.1)
    reduce_llm_chain = LLMChain(llm=llm, prompt=REDUCE_PROMPT)

    # Takes a list of documents and combines them into a single string
    combine_documents_chain = StuffDocumentsChain(
            llm_chain=reduce_llm_chain,
            document_variable_name="text_summaries")

    # Combines and iteravely reduces the mapped documents
    reduce_documents_chain = ReduceDocumentsChain(
        # This is final chain that is called.
        combine_documents_chain=combine_documents_chain,
        # If documents exceed context for `combine_documents_chain`
        collapse_documents_chain=combine_documents_chain,
        # The maximum number of tokens to group documents into.
        token_max=15000)

    # Combining documents by mapping a chain over them, then combining results
    combine_documents = MapReduceDocumentsChain(
        # Map chain
        llm_chain=map_llm_chain,
        # Reduce chain
        reduce_documents_chain=reduce_documents_chain,
        # The variable name in the llm_chain to put the documents in
        document_variable_name="text",
        # Return the results of the map steps in the output
        return_intermediate_steps=False)

    # Define Map=Reduce
    map_reduce = MapReduceChain(
        # Chain to combine documents
        combine_documents_chain=combine_documents,
        # Splitter to use for initial split
        text_splitter=text_splitter)

    return map_reduce.run(input_text=input_doc)

In [10]:
map_template_string = """Context: {text}
As an expert legal analyst, review the Context which is part of an oral argument session and do three things:

1. Identify the key facts of the case.
2. Summarize plaintiff counsel's main arguments.
3. Summarize rebuttals to plaintiff counsel's arguments.
"""

reduce_template_string = template = """Context: {text_summaries}
As an experienced legal analyst, use the context to:

Compose a case brief using only the Context.
Working step by step, organize the Context into a well-structured
outline with the following sections:

Facts of Loss: a concise sumary of facts and events relevant to this legal case.
Issue: [the facts of the case being argued]
Analysis: arguments
   ["Argument 1": key points from a distinct argument,
    "Rebuttal 1": key points from the rebuttal to argument 1,
    "Argument 2": key points from a distinct argument,
    "Rebuttal 2": key points from the rebuttal to argument 2,
    "Argument X": ...,
    "Rebuttal X": ...]
Takeaway: the most important argument assertion and its corresponding rebuttal.
TODO: optional list of knowledge gaps, or logical flaws in any of the arguments or their rebuttals.
"""
MAP_PROMPT = PromptTemplate(input_variables=["text"], template=map_template_string)
REDUCE_PROMPT = PromptTemplate(input_variables=["text_summaries"], template=reduce_template_string)

In [11]:
result = run_map_reduce(text, MAP_PROMPT, REDUCE_PROMPT)

In [12]:
display(Markdown(f"{result}"))

### Case Brief

**Title:** Kenneth Armani v. Illinois Department of Insurance et al.

**Facts of Loss:**
Kenneth Armani, the appellant, is self-represented in a legal dispute against the Illinois Department of Insurance and associated parties. Armani, who held a producer's license for over 20 years, claims that the Department wrongfully accused him of check fraud and revoked his license. He alleges that this revocation was retaliatory, following his complaints to the EEOC and the Illinois Department of Human Rights. Armani argues that the Department lacked authority and evidence to charge him with check fraud and that their actions have severely damaged his life and reputation. He seeks to reverse the dismissal of his case, reinstate his license, and obtain damages.

**Issue:**
Whether the Illinois Department of Insurance wrongfully revoked Kenneth Armani's producer's license on the grounds of check fraud without proper authority or evidence, and whether the revocation was retaliatory in nature.

**Analysis:**

**Argument 1:** Armani contends that the Department of Insurance had no legal basis to charge him with check fraud, as they provided no original documents or evidence, and that the charges were fabricated.

**Rebuttal 1:** Assistant Attorney General Laura O'Mara argues that the Department never criminally charged Armani with check fraud but found him engaged in dishonest practices under the civil provisions of the Illinois insurance code.

**Argument 2:** Armani asserts that the Department's actions were in retaliation for his prior complaints to the EEOC and the Illinois Department of Human Rights.

**Rebuttal 2:** O'Mara does not directly address the retaliation claim but focuses on the legitimacy of the administrative decisions and the separate proceedings related to Armani's license renewal.

**Argument 3:** Armani points to the Office of the Executive Inspector General's order for a rehearing as evidence that the Department acted without proper authority.

**Rebuttal 3:** O'Mara maintains that the circuit court lacked jurisdiction over Armani's action as it was an improper collateral attack on a final administrative decision.

**Takeaway:**
The most critical argument is Armani's claim that the Department had no legal basis to charge him with check fraud, which is directly countered by O'Mara's assertion that the Department found Armani engaged in dishonest practices under the Illinois insurance code, not criminal check fraud.

**TODO:**
- Verify the details of the Office of the Executive Inspector General's order for a rehearing and its implications on the case.
- Clarify whether there is any evidence supporting Armani's retaliation claims.
- Assess the validity of the jurisdiction argument presented by O'Mara and whether Armani's action is indeed an improper collateral attack on a final administrative decision.
- Examine the specifics of the 2020 proceedings related to Armani's license renewal and the evidence of selling insurance policies during his suspension.
- Determine the applicability of sovereign immunity in this case and whether Armani's claims for malicious prosecution or slander are barred by it.

In [2]:
class CourtroomParty(BaseModel):
    """Identifying information about a person."""
    
    person: str = Field(..., description="The name of the person speaking")
    role: Optional[str] = Field(None, description="The role of the person speaking")
    background: Optional[str] = Field(None, description="A concise summary about the person")
    

class OralArgument(BaseModel):
    """Identifying information about a particular argument."""
    
    topic: str = Field(..., description="A concise topic title for the argument")
    side: Optional[str] = Field(None, description="The side the argument favors, plaintiff or defense")
    person: Optional[str] = Field(None, description="The name of the person presenting the argument")
    summary: Optional[str] = Field(None, description="A concise summary of key points")
    

class OralDocket(BaseModel):
    """Identifying information in a text."""

    parties: Sequence[CourtroomParty] = Field(..., description="The information of interest in the text")
    arguments: Sequence[OralArgument] = Field(..., description="The information of interest in the text")

In [8]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 3040,
    chunk_overlap  = 0,
    is_separator_regex = False,
)

In [16]:
loader = TextLoader("data/transcriptions/MannievIllinoisDepartmentofInsurance.txt", encoding='utf-8')
doc = loader.load()

In [10]:
docs = text_splitter.split_documents(doc)

In [48]:
llm = ChatOpenAI(model="gpt-4-1106-preview", temperature=0)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a world class algorithm for extracting information in structured formats.",
        ),
        (
            "human",
            "Use the given format to extract information from the following input: {input}",
        ),
        ("human", "Tip: Make sure to answer in the correct format"),
    ]
)

In [26]:
runnable = create_structured_output_runnable(OralDocket, llm, prompt)

In [27]:
res_doc = runnable.invoke({"input": doc})

In [28]:
import pandas as pd
from src.types import Document
from src.language_models.openai_gpt import OpenAIGPT
from src.language_models.openai_gpt import OpenAIGPTConfig
from src.utils.pydantic_utils import dataframe_to_documents
from src.tools.segment_extract_tool  import SegmentExtractTool
from src.utils.utils import dataframe_to_lc_documents
from src.utils.pydantic_utils import flatten_pydantic_instance
from src.utils.utils import dataframe_summary

In [2]:
df = pd.read_parquet("data/reddit_legal_cluster_test_results.parquet")

In [29]:
lc_docs = dataframe_to_lc_documents(df, content='body', metadata=['title', 'text_label', 'State'])

In [10]:
test_query = df['body'].head(1).tolist()[0]
print(test_query)

I applied for a job and after two interviews I was given for and agreed to an offer letter, had a start date.  The only remaining variable was a drug test, which I prepared myself for by bringing all my prescriptions with me.  

I take Adderall, my doctor prescribes me for 3 pills a day, insurance pays for only two a day so that’s what I get.  2 pills a day for 30 days.  

However, I don’t need adderall everyday, so I only take it as needed.  My psychiatrist knows this and is okay with it.  I don’t take it everyday so my prescription can last me months, depending on circumstances.  

Of course, I happen to take a drug test for a possible new job the day after taking an adderall at work.  No big deal, I have a prescription and it’s never been a big deal in the past.  

Except this time it was.  According to the doctor who administrated the test, my prescription was too old.  How old was it, I was prescribed in June and filled the prescription on July 8th, 2019.  I still have about a thi

In [4]:
from typing import List
from pydantic import BaseModel, Field
from src.utils.pydantic_utils import flatten_pydantic_instance
from src.utils.utils import dataframe_to_lc_documents
import asyncio

class LegalIssue(BaseModel):
    """Identifying information about a legal issue."""
    topic: str = Field(..., description="A concise topic title for the legal issue")
    summary: Optional[str] = Field(None, description="A concise summary of key points")

class LegalIssues(BaseModel):
    """Identifying information in a text."""
    issues: Sequence[LegalIssue] = Field(..., description="The information of interest in the text")

async def extract_legal_issues(df: pd.DataFrame, content: str = 'body', metadata: List[str] = None, model_name: str = "gpt-4-1106-preview", temperature: float = 0.0) -> pd.DataFrame:
    """
    Extracts legal issues from a dataframe using a language model.

    Args:
    df (pd.DataFrame): The dataframe to extract legal issues from.
    content (str): The column containing the text content.
    metadata (list): List of columns to use for metadata.
    model_name (str, optional): The name of the language model to use. Defaults to "gpt-4-1106-preview".
    temperature (float, optional): The temperature to use for the language model. Defaults to 0.0.

    Returns:
    pd.DataFrame: A dataframe containing the extracted legal issues.
    """
    lc_docs = dataframe_to_lc_documents(df, content, metadata)

    llm = ChatOpenAI(model=model_name, temperature=temperature)

    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are a world class algorithm for extracting information in structured formats.",
            ),
            (
                "human",
                "Use the given format to extract information from the following input: {input}",
            ),
            ("human", "Tip: Make sure to answer in the correct format"),
        ]
    )

    runnable = create_structured_output_runnable(LegalIssues, llm, prompt)

    results = []
    for doc in lc_docs:
        res_doc = await runnable.ainvoke({"input": doc})
        res_doc_flat = flatten_pydantic_instance(res_doc, "")
        df = pd.DataFrame(res_doc_flat['issues'])
        df['id_'] = doc.metadata['id_']
        results.append(df)

    return pd.concat(results, ignore_index=True)

In [5]:
df_sample = df.sample(10)

In [6]:
test_res = await extract_legal_issues(df_sample)

In [71]:
test_res

,topic,summary,id_
0,Non-Compete Clause Interpretation,The non-compete clause in the contract with Dr...,2111
1,Competitor Definition,The YouTube channel offering fantasy football ...,2111
2,Enforceability of Non-Compete Agreements,The enforceability of non-compete agreements v...,2111
3,First Amendment and Criminal Intent,The discussion revolves around the interpretat...,4373
4,Traffic Ticket Without Recorded Speed,The absence of a recorded speed on a traffic t...,1418
5,Tenant Rights and Responsibilities,"A group of tenants, who are strangers to each ...",6857
6,Financial Hardship Due to COVID-19,One roommate's financial hardship due to job l...,6857
7,Lease Termination and Buyout Options,The tenant in financial hardship cannot afford...,6857
8,Property Damage in Parking Garage,A car owner parked in a self-service garage an...,51
9,Potential Online Scam,The individual may be involved in a potential ...,9334


In [9]:
test_res.id_.nunique()

10

In [12]:
lc_docs = dataframe_to_lc_documents(test_res, 'summary')

In [17]:
import numpy as np
from sentence_transformers import CrossEncoder
from typing import List
from langchain.schema import Document

def rerank_with_cross_encoder(
    query: str, passages: List[Document], n_similar_docs: int
) -> List[Document]:

    model = CrossEncoder("BAAI/bge-reranker-large")
    scores = model.predict([(query, p.page_content) for p in passages])
    # Convert to [0,1] so we might could use a cutoff later.
    scores = 1.0 / (1 + np.exp(-np.array(scores)))
    # get top k scoring passages
    sorted_pairs = sorted(
        zip(scores, passages),
        key=lambda x: x[0],
        reverse=True,
    )
    passages = [
        d for _, d in sorted_pairs[: n_similar_docs]
    ]
    return passages

In [18]:
reranked = rerank_with_cross_encoder(test_query, lc_docs, 10)

In [19]:
reranked

[Document(page_content="A group of tenants, who are strangers to each other, signed a lease to take over someone else's lease. The management company's practices, such as requiring deposits to be sent via Venmo to previous tenants and not ensuring the property was cleaned before move-in, were questionable. The tenants are jointly responsible for the lease, and one tenant's inability to pay due to job loss from the coronavirus pandemic has led to the management company demanding the remaining roommates cover the shortfall. This situation raises concerns about tenant rights, joint and several liability, and the responsibilities of property management companies during a crisis.", metadata={'id_': 5}),
 Document(page_content="The non-compete clause in the contract with Draft Kings prohibits employment with any entity in the online fantasy sports contests business. Whether an unpaid position with a fantasy football advice YouTube channel constitutes 'entering employ' under this clause is un

In [65]:
from src.parsing.utils import number_segments, extract_numbered_segments, parse_number_range_list, get_specs

In [25]:
numbered_passage = number_segments(test_query, 4)

In [67]:
from langchain.chains import LLMChain

def extract_relevant(query: Optional[str] = None, message: Optional[str] = None, segment_length: int = 1):
    model = ChatOpenAI(model="gpt-4-1106-preview", temperature=0.0)
    # number the segments in the passage
    numbered_passage = number_segments(message, segment_length)
    # build prompt
    prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "The user will give you a PASSAGE containing segments numbered as <#1#>, <#2#>, <#3#>, etc., followed by a QUERY. Extract ONLY the segment-numbers from the PASSAGE that are RELEVANT to the QUERY.",
        ),
        (
            "human",
            "PASSAGE:\n{numbered_passage}\n\nQUERY:\n{query}",
        ),
    ]
    )

    chain = LLMChain(llm=model, prompt=prompt)
    spec = chain.run(query=query, numbered_passage=numbered_passage)
    specs = get_specs(spec)
    extracts = extract_numbered_segments(numbered_passage, specs)
    return extracts

In [32]:
test_query

'I applied for a job and after two interviews I was given for and agreed to an offer letter, had a start date.  The only remaining variable was a drug test, which I prepared myself for by bringing all my prescriptions with me.  \n\nI take Adderall, my doctor prescribes me for 3 pills a day, insurance pays for only two a day so that’s what I get.  2 pills a day for 30 days.  \n\nHowever, I don’t need adderall everyday, so I only take it as needed.  My psychiatrist knows this and is okay with it.  I don’t take it everyday so my prescription can last me months, depending on circumstances.  \n\nOf course, I happen to take a drug test for a possible new job the day after taking an adderall at work.  No big deal, I have a prescription and it’s never been a big deal in the past.  \n\nExcept this time it was.  According to the doctor who administrated the test, my prescription was too old.  How old was it, I was prescribed in June and filled the prescription on July 8th, 2019.  I still have ab

In [68]:
query = "How many pills does this person take?"

res = extract_relevant(query, test_query, 1)

In [70]:
print(res)

I take Adderall, my doctor prescribes me for 3 pills a day, insurance pays for only two a day so that’s what I get.  2 pills a day for 30 days.

However, I don’t need adderall everyday, so I only take it as needed.  I don’t take it everyday so my prescription can last me months, depending on circumstances.

I still have about a third of a bottle, I take adderall twice a week on average. 


In [56]:
res_doc_flat = flatten_pydantic_instance(res_doc, "")
print(res_doc_flat)

{'parties': [{'person': 'Kenneth Armani', 'role': 'Senior Appellant', 'background': 'Charged with check fraud by the Illinois Department of Insurance, which he claims was fabricated in retaliation for his complaints to the EEOC and Department of Human Rights.'}, {'person': 'Martin Grovy', 'role': 'Appellee', 'background': 'Representative of the Illinois Department of Insurance.'}, {'person': 'Helen Kim', 'role': 'Appellee', 'background': 'Representative of the Illinois Department of Insurance.'}, {'person': 'Patrick Riley', 'role': 'Appellee', 'background': 'Representative of the Illinois Department of Insurance.'}, {'person': 'Laura Omera', 'role': 'Assistant Attorney General', 'background': "Representing the Illinois Department of Insurance, defending the department's decision to suspend Kenneth Armani's license."}], 'arguments': [{'topic': 'Check Fraud Charge', 'side': 'Plaintiff', 'person': 'Kenneth Armani', 'summary': 'Argues that the check fraud charge was fabricated by the Illin

In [57]:
pd.DataFrame(res_doc_flat['parties'])

,person,role,background
0,Kenneth Armani,Senior Appellant,Charged with check fraud by the Illinois Depar...
1,Martin Grovy,Appellee,Representative of the Illinois Department of I...
2,Helen Kim,Appellee,Representative of the Illinois Department of I...
3,Patrick Riley,Appellee,Representative of the Illinois Department of I...
4,Laura Omera,Assistant Attorney General,Representing the Illinois Department of Insura...


In [50]:
df.drop("panel", inplace=True, axis=1)

In [52]:
Markdown(f"{dataframe_summary(df)}")

COLUMN NAMES:
'resource_uri' 'id' 'absolute_url' 'docket' 'date_created' 'date_modified' 'source' 'case_name_short' 'case_name' 'case_name_full' 'judges' 'sha1' 'download_url' 'local_path_mp3' 'local_path_original_file' 'filepath_ia' 'ia_upload_failure_count' 'duration' 'processing_complete' 'date_blocked' 'blocked' 'stt_status' 'stt_google_response' 'insurance_found'

Numerical Column Summary:
             id duration stt_status insurance_found
count    200.00   199.00     200.00          200.00
mean   89839.50  1708.59       0.00            0.02
std       57.88   684.27       0.00            0.14
min    89740.00   347.00       0.00            0.00
25%    89789.75  1220.50       0.00            0.00
50%    89839.50  1610.00       0.00            0.00
75%    89889.25  2015.00       0.00            0.00
max    89939.00  4080.00       0.00            1.00

Categorical Column Summary:
'resource_uri': 200 unique values
'absolute_url': 200 unique values
'docket': 200 unique values
'date_created': 200 unique values
'date_modified': 200 unique values
'source': C
'case_name_short': 121 unique values
'case_name': 199 unique values
'case_name_full': 
'judges': , Millett, Wilkins, Garcia, KOH, THOMAS, DESAI, TASHIMA, UNKNOWN, CHRISTEN, Millett, Pan, Randolph, GOULD, KOH, DESAI
'sha1': 200 unique values
'download_url': 200 unique values
'local_path_mp3': 199 unique values
'local_path_original_file': 200 unique values
'filepath_ia': 200 unique values
'ia_upload_failure_count': None
'date_blocked': None
'stt_google_response': 

Missing Values Column Summary:
                          missing_values
resource_uri                           0
id                                     0
absolute_url                           0
docket                                 0
date_created                           0
date_modified                          0
source                                 0
case_name_short                        0
case_name                              0
case_name_full                         0
judges                                 0
sha1                                   0
download_url                           0
local_path_mp3                         1
local_path_original_file               0
filepath_ia                            0
ia_upload_failure_count              200
duration                               1
processing_complete                    0
date_blocked                         200
blocked                                0
stt_status                             0
stt_google_response                    0
insurance_found                        0


In [146]:
class CourtListenerExtractor:
    def __init__(self, database_path, base_url="https://www.courtlistener.com/api/rest/v3/audio/"):
        """
        Initializes the CourtListenerExtractor with the path to the database file.

        Args:
        database_path (str): The path to the Excel file database.
        """
        self.database_path = database_path
        self.base_url=base_url

    def fetch_records(self, start_page, end_page):
        """
        Fetches records from Court Listener based on a page range.
        
        Args:
        start_page (int): court listener web search start
        end_page (int): court listener web search start

        Returns:
        pd.DataFrame: A DataFrame containing the fetched records.
        """
        all_results = []
        for page in tqdm(range(start_page, end_page + 1)):
            url = f"{self.base_url}?page={page}"
            data = self.fetch_page_(url)
            all_results.extend(data["results"])
            time.sleep(1)
        df = pd.DataFrame.from_records(all_results)
        return df

    def add_rows_to_excel(self, input_df):
        # Convert lists to strings in input_df
        input_df = input_df.applymap(lambda x: str(x) if isinstance(x, list) else x)
        
        existing_df = pd.DataFrame()
        if os.path.exists(self.database_path):
            # Load the existing data
            existing_df = pd.read_excel(self.database_path)

            # Convert lists to strings in existing_df
            existing_df = existing_df.applymap(lambda x: str(x) if isinstance(x, list) else x)

        # Concatenate the existing data with the new data
        combined_df = pd.concat([existing_df, input_df])

        # Remove the 'unique_index' column before removing duplicates
        combined_df = combined_df.drop(columns=['unique_index'])

        # Remove duplicates
        final_df = combined_df.drop_duplicates()

        # Reassign the 'unique_index' column
        final_df['unique_index'] = range(1, len(final_df) + 1)

        print(f"Total number of rows: {len(final_df)}")
        print(f"Number of new rows added: {len(final_df) - len(existing_df)}")

        # Write the data to the Excel file
        final_df.to_excel(self.database_path, index=False)

    def update_database(self, new_records):
        """
        Updates the database with new records.

        Args:
        new_records (pd.DataFrame): New records to be added to the database.
        """
        self.add_rows_to_excel(new_records)
        
    @backoff.on_exception(backoff.expo, (requests.exceptions.RequestException, requests.exceptions.HTTPError), max_tries=8)
    def fetch_page_(self, url):
        response = requests.get(url)
        response.raise_for_status()  # Raises a HTTPError if the status is 4xx, 5xx
        return response.json()
    
    def find_subset_with_keyword(self, keyword="insurance"):
        """
        Finds a subset of records containing a specific keyword.

        Args:
        keyword (str, optional): The keyword to search for. Defaults to "insurance".

        Returns:
        pd.DataFrame: A DataFrame containing the subset of records with the keyword.
        """
        if not os.path.exists(self.database_path):
            raise FileNotFoundError(f"Database file {self.database_path} not found.")

        # Load the existing data
        df = pd.read_excel(self.database_path)

        # Finding the subset with the keyword
        subset_df = df[df.apply(lambda row: row.astype(str).str.contains(keyword).any(), axis=1)]
        print(f"Records with keyword: {keyword}: {len(subset_df)}")
        
        return subset_df

# Example usage:
# extractor = CourtListenerExtractor("path/to/database.xlsx")
# new_records = extractor.fetch_records(1, 11)
# extractor.update_database(new_records)

In [129]:
extractor = CourtListenerExtractor("database.xlsx")

In [130]:
new_records = extractor.fetch_records(1, 6)

100%|██████████| 6/6 [00:08<00:00,  1.48s/it]


In [131]:
new_records.head(2)

,resource_uri,id,absolute_url,panel,docket,date_created,date_modified,source,case_name_short,case_name,case_name_full,judges,sha1,download_url,local_path_mp3,local_path_original_file,filepath_ia,ia_upload_failure_count,duration,processing_complete,date_blocked,blocked,stt_status,stt_google_response
0,https://www.courtlistener.com/api/rest/v3/audi...,89173,/audio/89173/appvion-inc-retirement-savin-v-do...,[],https://www.courtlistener.com/api/rest/v3/dock...,2023-12-01T14:30:43.486459-08:00,2023-12-01T14:30:43.486469-08:00,C,,"Appvion, Inc. Retirement Savin v. Douglas Buth",,,f2922a7f6a7085a3a6047ab6cd64e73d5e2d2180,http://media.ca7.uscourts.gov/sound/external/k...,mp3/2023/12/01/appvion_inc._retirement_savin_v...,mp3/2023/12/01/appvion_inc._retirement_savin_v...,,None,3738.0,True,None,False,0,
1,https://www.courtlistener.com/api/rest/v3/audi...,89172,/audio/89172/denise-evans-v-united-states/,[],https://www.courtlistener.com/api/rest/v3/dock...,2023-12-01T14:30:37.142049-08:00,2023-12-01T14:30:37.142060-08:00,C,,Denise Evans v. United States,,,37fa42c525b70a19c7ededd868cb9219d0bac5c8,http://media.ca7.uscourts.gov/sound/external/k...,mp3/2023/12/01/denise_evans_v._united_states_c...,mp3/2023/12/01/denise_evans_v._united_states.mp3,,None,1470.0,True,None,False,0,


In [132]:
extractor.update_database(new_records)

Total number of rows: 340
Number of new rows added: 120


In [145]:
class AudioTranscriber:
    def __init__(self, download_path, text_output_path, model_name='base'):
        self.download_path = download_path
        self.text_output_path = text_output_path
        self.transcription_model = whisper.load_model(model_name)
        self._create_directory_if_not_exists(download_path)
        self._create_directory_if_not_exists(text_output_path)

    def _create_directory_if_not_exists(self, path):
        if not os.path.exists(path):
            os.makedirs(path)

    def download_mp3(self, url, file_name, unique_index):
        file_path = os.path.join(self.download_path, f"{file_name}_{unique_index}.mp3")
        if os.path.exists(file_path):
            print(f"File {file_name}.mp3 already exists.")
            return file_path

        try:
            response = requests.get(url, stream=True)
            response.raise_for_status()

            with open(file_path, 'wb') as file:
                for chunk in response.iter_content(chunk_size=8192):
                    file.write(chunk)

            print(f"File downloaded and saved as {file_path}")
            return file_path
        except requests.RequestException as e:
            print(f"An error occurred: {e}")

    def transcribe_audio(self, file_path):
        result = self.transcription_model.transcribe(file_path)
        return result['text']
    
    def save_transcription(self, text, file_name, unique_index):
        text_file_path = os.path.join(self.text_output_path, f"{file_name}_{unique_index}.txt")
        with open(text_file_path, 'w') as file:
            file.write(text)
        print(f"Transcription saved as {text_file_path}")

    def read_transcription(self, file_name, unique_index):
        text_file_path = os.path.join(self.text_output_path, f"{file_name}_{unique_index}.txt")
        if not os.path.exists(text_file_path):
            print(f"No transcription file found for {file_name}")
            return None
        with open(text_file_path, 'r') as file:
            return file.read()

    def process_batch(self, df):
        for _, row in df.iterrows():
            url = row['download_url']
            name = re.sub(r'\W+', '', row['case_name'])
            unique_index = row['unique_index']
            file_path = self.download_mp3(url, name, unique_index)
            if file_path:
                transcription = self.transcribe_audio(file_path)
                self.save_transcription(transcription, name, unique_index)

# Example usage:
# audio_transcriber = AudioTranscriber('path/to/download', 'path/to/text_output')
# extractor = CourtListenerExtractor("database.xlsx")
# df = extractor.fetch_records(1, 11)
# audio_transcriber.process_batch(df)

In [137]:
audio_transcriber = AudioTranscriber('audio', 'transcriptions')

In [138]:
extractor = CourtListenerExtractor("database.xlsx")

In [ ]:
df = extractor.fetch_records(1, 11)

In [139]:
insurance_cases = extractor.find_subset_with_keyword()

In [141]:
insurance_cases.head(2)

,resource_uri,id,absolute_url,panel,docket,date_created,date_modified,source,case_name_short,case_name,case_name_full,judges,sha1,download_url,local_path_mp3,local_path_original_file,filepath_ia,ia_upload_failure_count,duration,processing_complete,date_blocked,blocked,stt_status,stt_google_response,unique_index
16,https://www.courtlistener.com/api/rest/v3/audi...,89157,/audio/89157/mannie-v-illinois-department-of-i...,[],https://www.courtlistener.com/api/rest/v3/dock...,2023-11-30T14:54:57.601458-08:00,2023-12-01T01:38:54.934186-08:00,C,Mannie,Mannie v. Illinois Department of Insurance,NaN,NaN,efc0ffa7dcce6495ee918087d7d986f520f89168,https://ilcourtsaudio.blob.core.windows.net/an...,mp3/2023/11/30/mannie_v._illinois_department_o...,mp3/2023/11/30/mannie_v._illinois_department_o...,https://archive.org/download/gov.uscourts.illa...,NaN,1212.0,True,NaN,False,0,NaN,17
18,https://www.courtlistener.com/api/rest/v3/audi...,89155,/audio/89155/till-v-national-health-insurance-co/,[],https://www.courtlistener.com/api/rest/v3/dock...,2023-11-30T14:54:52.407621-08:00,2023-12-01T02:00:48.933627-08:00,C,Till,Till v. National Health Insurance Co.,NaN,NaN,e4cceca79c20f86832d1eb77c122bfceb6597a77,https://ilcourtsaudio.blob.core.windows.net/an...,mp3/2023/11/30/till_v._national_health_insuran...,mp3/2023/11/30/till_v._national_health_insuran...,https://archive.org/download/gov.uscourts.illa...,NaN,2153.0,True,NaN,False,0,NaN,19


In [142]:
%%time
audio_transcriber.process_batch(insurance_cases.head(2))

File downloaded and saved as audio\MannievIllinoisDepartmentofInsurance.mp3
Transcription saved as transcriptions\MannievIllinoisDepartmentofInsurance.txt
File downloaded and saved as audio\TillvNationalHealthInsuranceCo.mp3
Transcription saved as transcriptions\TillvNationalHealthInsuranceCo.txt
CPU times: total: 41min 45s
Wall time: 5min 23s


In [150]:
class OralDocketExtractor:
    def __init__(self, model_name="gpt-4-1106-preview"):
        self.model = ChatOpenAI(model=model_name, temperature=0.1)

    def process_document(self, document_path: str) -> OralDocket:
        loader = TextLoader(document_path)
        doc = loader.load()
            
        prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are a world class algorithm for extracting information in structured formats.",
            ),
            (
                "human",
                "Use the given format to extract information from the following input: {input}",
            ),
            ("human", "Tip: Make sure to answer in the correct format"),
        ]
    )

        runnable = create_structured_output_runnable(OralDocket, self.model, prompt)
        result = runnable.invoke({"input": doc})
        
        # Extract unique_index from the filename
        filename = os.path.basename(document_path)
        unique_index = filename.split("_")[-1].split(".")[0]
        
        return result, unique_index
    
    def save_to_excel(self, data: List[OralDocket], file_name: str, unique_index: int):
        data_dicts = [instance.dict() for instance in data]

        # Save 'parties' and 'arguments' to separate sheets
        for key in ['parties', 'arguments']:
            df_new = pd.concat([pd.json_normalize(data_dict[key]) for data_dict in data_dicts], ignore_index=True)
            df_new['unique_index'] = unique_index  # Add the unique_index column
            self.write_to_excel(df_new, key, file_name)

    @staticmethod
    def write_to_excel(df_new, sheet_name, file_name='court_whisperer_data.xlsx'):
        if os.path.exists(file_name):
            try:
                # Attempt to read the existing sheet
                df_old = pd.read_excel(file_name, sheet_name=sheet_name)
                df = pd.concat([df_old, df_new]).drop_duplicates()
            except ValueError:
                # If the sheet does not exist, use the new dataframe as is
                df = df_new

            # Open the existing workbook using 'a' mode (append)
            with pd.ExcelWriter(file_name, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
                # If the sheet exists, it will be overwritten
                df.to_excel(writer, sheet_name=sheet_name, index=False)
        else:
            # Create a new file if it doesn't exist
            df_new.to_excel(file_name, sheet_name=sheet_name, index=False)


In [147]:
# Example usage:
audio_transcriber = AudioTranscriber('audio', 'transcriptions')
extractor = CourtListenerExtractor("database.xlsx")
df = extractor.fetch_records(1, 21)
insurance_cases = extractor.find_subset_with_keyword()
audio_transcriber.process_batch(insurance_cases.tail(2))

100%|██████████| 21/21 [00:31<00:00,  1.50s/it]


Records with keyword: insurance: 11
File downloaded and saved as audio\MariaJimenezvTravelersCommercialInsurance_249.mp3
Transcription saved as transcriptions\MariaJimenezvTravelersCommercialInsurance_249.txt
File downloaded and saved as audio\NationalFireInsuranceCovSanchez_282.mp3
Transcription saved as transcriptions\NationalFireInsuranceCovSanchez_282.txt


In [152]:
llm_processor = OralDocketExtractor()

In [153]:
text_files = [
    "MariaJimenezvTravelersCommercialInsurance_249.txt",
    "NationalFireInsuranceCovSanchez_282.txt"
]

for text_file in text_files:
    result, unique_index = llm_processor.process_document(f"transcriptions/{text_file}")
    llm_processor.save_to_excel([result], "output_file_v2.xlsx", unique_index)

In [154]:
df_parties = pd.read_excel("output_file_v2.xlsx", sheet_name="parties")
df_arguments = pd.read_excel("output_file_v2.xlsx", sheet_name="arguments")

In [155]:
print(f"Shape of parties df: {df_parties.shape}")
Markdown(f"{df_parties.to_markdown(index=False)}")

Shape of parties df: (10, 4)


| person                         | role                 | background                                                                                                                                                                                                                                                      |   unique_index |
|:-------------------------------|:---------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|---------------:|
| Mr. Keifer                     | Defendant            | Resident of Illinois, involved in a legal dispute with Travelers Commercial Insurance regarding the removal of a state court citation to discover assets. An agreement was made with the plaintiffs for a judgment and a stipulation not to execute in Florida. |            249 |
| Travelers Commercial Insurance | Defendant/Appellee   | Insurance company from Connecticut, involved in a legal dispute over the removal of a state court citation to discover assets and a subsequent answer under oath stating no property or monies belonging to the judgment debtor.                                |            249 |
| Mr. Bustamante                 | Plaintiff's Counsel  | Representing the plaintiffs, arguing that the removal was improper and there was no independent controversy at the time of removal. Contends that the case should not have been removed and that the removal violated the forum defendant rule.                 |            249 |
| Mr. Valentino                  | Defendant's Counsel  | Representing Travelers Commercial Insurance, arguing for the affirmation of the district court's decisions and maintaining that the removal of the state court citation to discover assets was appropriate.                                                     |            249 |
| Margaret Esmechbriah           | Justice              | Presiding over the case National Fire Insurance Company versus Lewis Sanchez.                                                                                                                                                                                   |            282 |
| Ellis                          | Justice              | nan                                                                                                                                                                                                                                                             |            282 |
| Cops                           | Justice              | nan                                                                                                                                                                                                                                                             |            282 |
| Brad Stavis                    | Appellant's Attorney | Represents the appellant Mr. Sanchez.                                                                                                                                                                                                                           |            282 |
| Todd Shank                     | Appellee's Attorney  | On behalf of Appellee East Continental Insurance Company and National Fire Insurance Company of Hartford.                                                                                                                                                       |            282 |
| Lewis Sanchez                  | Appellant            | Represented a class of individuals whose biometric information was collected and used by Visual Pack and Elite Staffing.                                                                                                                                        |            282 |

In [156]:
print(f"Shape of arguments df: {df_arguments.shape}")
Markdown(f"{df_arguments.to_markdown(index=False)}")

Shape of arguments df: (10, 5)


| topic                                           | side      | person         | summary                                                                                                                                                                                                                                                                                                                                                            |   unique_index |
|:------------------------------------------------|:----------|:---------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|---------------:|
| Improper Removal                                | Plaintiff | Mr. Bustamante | Argues that the removal of the case was improper as there was no independent controversy to adjudicate at the time of removal, and it violated the forum defendant rule.                                                                                                                                                                                           |            249 |
| No Independent Controversy                      | Plaintiff | Mr. Bustamante | Contends that there was no actual independent controversy issue at the time of removal since Travelers had not filed an answer to the citation, preventing the district court from determining if there was an independent controversy.                                                                                                                            |            249 |
| Answer Under Oath                               | Plaintiff | Mr. Bustamante | Points out that Travelers' answer under oath stated it had no property or monies belonging to the judgment debtor, which should have ended the citation proceeding.                                                                                                                                                                                                |            249 |
| Forum Defendant Rule                            | Plaintiff | Mr. Bustamante | Asserts that removal was also improper under the forum defendant rule because Mr. Keifer, a defendant in the case, is a resident of Illinois.                                                                                                                                                                                                                      |            249 |
| Defense of Proper Removal                       | Defense   | Mr. Valentino  | Defends the removal of the case, stating that the district court correctly applied relevant decisions and that the citation to discover assets presents a separate action requiring its own basis for federal jurisdiction.                                                                                                                                        |            249 |
| Summary Judgment Appropriateness                | Defense   | Mr. Valentino  | Argues that it was appropriate for the judge to rule by summary judgment in the citation proceeding and that the plaintiffs failed to establish a factual basis for a bad faith claim against Travelers.                                                                                                                                                           |            249 |
| No Bad Faith Claim                              | Defense   | Mr. Valentino  | Maintains that the plaintiffs cannot establish a bad faith claim against Travelers as they failed to satisfy the necessary elements and did not provide evidence of Travelers acting in bad faith.                                                                                                                                                                 |            249 |
| Application of Insurance Policy to BIPA Claim   | Appellant | Brad Stavis    | The case involves the application of an insurance policy to a claim brought under the Biometric Information Privacy Act (BIPA). The main issues are whether estoppel applies, preventing the insurance company from asserting exclusions in the policy, and whether there is coverage under the policy based on the West Bend versus Krzysztof Supreme Court case. |            282 |
| Estoppel and Exclusions in Insurance Policy     | Appellant | Brad Stavis    | Argues that estoppel applies due to the insurance company's delay in action, which should prevent them from asserting any exclusions. Even if estoppel does not apply, the violation of law exclusion, which is central to the case, is argued to be ambiguous and should not prevent coverage.                                                                    |            282 |
| BIPA Violations and Insurance Policy Exclusions | Appellee  | Todd Shank     | Claims that estoppel cannot apply unless there is a breach of duty to defend, which is not the case here due to clear exclusions that apply to the BIPA violations in the Sanchez lawsuit. Argues against the ambiguity of the catch-all in the violation of law exclusion and supports the trial court's judgment in favor of CNA.                                |            282 |

In [61]:
document_path = "MariaJimenezvTravelersCommercialInsurance_249.txt"

loader = TextLoader(document_path)
doc = loader.load()

embeddings = OpenAIEmbeddings()
db = Chroma.from_documents(docs, embeddings)

retriever = db.as_retriever()

template = """Answer the question based only on the following context:

{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI(model="gpt-4-1106-preview", temperature=0)


def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])


chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [62]:
result = chain.invoke("Generate a well strucured list of legal arguments.")

In [63]:
Markdown(f"{result}")

Based on the provided context, here is a well-structured list of legal arguments that appear to be relevant to the case:

1. **Breach of Contract Argument:**
   - The argument that there is no breach of contract by Southwest because the contract allows for various ways Southwest can perform under the agreement.

2. **Definition of Loss Argument:**
   - The costs incurred that are covered by the first definition of loss are not limited to liabilities but also include voluntary decisions made by the policyholder as a result of the system failure.

3. **Excessive Costs Argument:**
   - The carrier has the right to argue that certain costs were excessive and not solely motivated by the system failure, and this can be presented to the fact-finder.

4. **Historical Practice Argument:**
   - Southwest's actions were consistent with its historical practices, and the benefits provided were in line with what it has historically used.

5. **Comparative Value Argument:**
   - The per passenger value of the benefits provided by Southwest was less than that provided by Delta in a similar situation, suggesting that the costs were reasonable.

6. **Voluntary Decision Argument:**
   - The policy covers voluntary decisions to incur negative consequences, as indicated by the etymology of the word "incur."

7. **Irrelevance of the Seventh Circuit Decision Argument:**
   - The Seventh Circuit decision is immaterial to the case at hand as it dealt with a passenger's contract claim, not the insurance policy coverage.

8. **Consequential Damages Argument:**
   - The policy is presumed to exclude consequential damages that are not directly related to the covered occurrence.

9. **First Impression Argument:**
   - This case is of first impression, meaning there is no existing precedent directly on point, and the court's decision will set a new precedent.

10. **Mitigation Costs Argument:**
    - Costs incurred by a business to mitigate further losses due to a covered occurrence should be considered a covered loss under the policy.

11. **Contractual Obligation Argument:**
    - If certain actions are required by the contract of carriage (e.g., taking care of passengers affected by a system failure), they may be excluded from coverage under the liability to third parties exclusion.

12. **Policy Exclusions Argument:**
    - Exclusions in the policy serve to narrow down coverage and should be interpreted to subtract from coverage, not to render the policy illusory.

13. **Policy Interpretation Argument:**
    - The plain language of the policy should guide the interpretation, and the court should adhere to the clear meaning of the terms used in the policy.

These arguments reflect the various positions and interpretations of the insurance policy and contractual obligations at issue in the case.

In [64]:
result_summary = chain.invoke(f"Based on the following list of arguments, distill each side's most important points:\n\n{result}")

In [65]:
Markdown(f"{result_summary}")

Based on the provided context, here are the most important points distilled from each side's arguments:

**Southwest's Side:**

1. **Breach of Contract Argument:**
   - Southwest contends that there is no breach of contract as the contract allows for various performance methods.

2. **Definition of Loss Argument:**
   - Southwest argues that the definition of loss includes both liabilities and voluntary decisions made due to system failure.

3. **Historical Practice Argument:**
   - Southwest maintains that its mitigation actions were consistent with historical practices, suggesting they were reasonable and should be covered.

4. **Voluntary Decision Argument:**
   - The policy is argued to cover voluntary decisions to incur negative consequences, which is consistent with the etymology and common understanding of "incur."

5. **Irrelevance of the Seventh Circuit Decision Argument:**
   - Southwest asserts that the Seventh Circuit decision is not relevant to the current case, which focuses on insurance policy coverage.

6. **Mitigation Costs Argument:**
   - Southwest believes that costs incurred to mitigate further losses due to a covered occurrence are covered losses under the policy.

**Insurance Carrier's Side:**

1. **Consequential Damages Argument:**
   - The carrier argues that the policy excludes consequential damages that are not directly related to the covered occurrence.

2. **First Impression Argument:**
   - The carrier notes that the case is of first impression and will set a new precedent, implying the need for careful consideration of the policy's terms.

3. **Contractual Obligation Argument:**
   - The carrier suggests that actions required by the contract of carriage may be excluded from coverage under the liability to third parties exclusion.

4. **Policy Exclusions Argument:**
   - The carrier emphasizes that exclusions in the policy are meant to narrow coverage and should not be interpreted to render the policy illusory.

5. **Policy Interpretation Argument:**
   - The carrier insists on a plain language interpretation of the policy, adhering to the clear meaning of the terms used.

Both sides present arguments related to the interpretation of the insurance policy, the definition of loss, and the relevance of contractual obligations and historical practices. The court's decision will hinge on the interpretation of these arguments and how they apply to the policy in question.

In [66]:
plaintiff_summary = chain.invoke("Craft a summary of the plaintiff's most key arguments in logical step-by-step fashion.")

In [67]:
Markdown(f"{plaintiff_summary}")

The plaintiff's key arguments can be summarized in a logical step-by-step fashion as follows:

1. **Issue of Sole Causation**: The plaintiff argues that the central question of the case is whether expenses incurred voluntarily by a policyholder to mitigate the impact of a covered business interruption can be considered as incurred solely as a result of the interruption. The plaintiff contends that this is typically a factual matter to be determined by a jury.

2. **Summary Judgment Challenge**: The plaintiff challenges the district court's summary judgment, which was based on the premise that voluntary mitigation expenses cannot be solely caused by a system failure due to the policyholder's business judgment being a concurrent cause. The plaintiff asserts that this judgment is erroneous.

3. **Policy Text and Precedent**: The plaintiff suggests that the case should be decided by looking at the policy text and by drawing analogies to prior court precedents, particularly when there is no existing case law directly addressing the policy language in question.

4. **Analogous Case - Wells vs. Minnesota Life Insurance**: The plaintiff cites the decision in Wells vs. Minnesota Life Insurance, where the court held that subsequent events stemming from an accidental injury do not necessarily sever the sole cause link for policy coverage. The plaintiff draws a parallel between this case and the current situation, arguing that the expenses were a direct result of the system failure.

5. **Evidence of Sole Causation**: The plaintiff points to evidence, including testimony from decision-makers at Southwest, affirming that the compensation decisions were made solely because of the system failure. This evidence is presented to support the argument that the expenses should be covered by the insurance policy.

6. **Nexus to System Failure**: The plaintiff addresses the concern about the nexus between the system failure and the compensation of individuals who did not travel during the interruption. The argument is that the compensation was linked to individual booking numbers, which could include up to eight passengers, thus establishing a connection to the system failure.

7. **Fact Finder's Role**: The plaintiff argues that it is the role of the fact finder to determine whether the compensation was excessive or beyond what was necessary due to the system failure. The plaintiff suggests that the insurer has the right to argue this point before the fact finder.

8. **Historical Business Practices**: The plaintiff asserts that the compensation provided was consistent with Southwest's historical practices, which is relevant to the policy's consideration of the insured's prior business experience.

9. **Definition of "Incurs"**: The plaintiff emphasizes the meaning of the term "incurs," arguing that it refers to a loss that an individual or entity brings upon itself, often voluntarily. This interpretation supports the plaintiff's view that the policy covers the expenses in question.

10. **Irrelevance of the Seven Circuit Decision**: Finally, the plaintiff dismisses a Seven Circuit decision cited by the opposition as immaterial, stating that it pertained to a passenger's contract claim against Southwest and not to the insurance policy coverage issue at hand.

The plaintiff concludes by requesting that the summary judgment be reversed, allowing the matter to be decided by a fact finder, as the evidence suggests that the expenses were indeed incurred solely as a result of the system failure.

In [70]:
defense_summary = chain.invoke("Using the following arguments, craft a concise, methodical, fact-focused counter argument as a bullet point list. Look for errors and omissions in the plaintiff logic:\n\n{plaintiff_summary}")
Markdown(f"{defense_summary}")

Counter Argument:

- **Voluntary Mitigation vs. Sole Causation:**
  - The plaintiff conflates voluntary mitigation efforts with expenses solely caused by the system failure. Voluntary actions may have multiple motivations beyond the immediate cause.
  - The policy language likely distinguishes between actions taken solely due to the interruption and those that are part of broader business strategies, including goodwill gestures.

- **Historical Practices Not Necessarily Relevant:**
  - Past practices of offering benefits do not automatically justify current expenses as being solely caused by the system failure.
  - The historical context provided by the plaintiff does not address whether the scale or scope of the mitigation efforts was proportionate to the interruption.

- **Excessive Compensation Claims:**
  - The plaintiff admits that the carrier could argue against excessive compensation, suggesting that there is a limit to what can be considered solely caused by the interruption.
  - The example of lifetime travel for goodwill exceeds reasonable mitigation and indicates that there is a line where voluntary decisions are no longer solely due to the system failure.

- **Fair Saver Program for Friends:**
  - The inclusion of friends in the compensation package lacks a direct nexus to the system failure, as they were not affected passengers.
  - The plaintiff's argument does not adequately address why non-traveling individuals should be compensated for an interruption they did not experience.

- **Advertising Costs:**
  - The plaintiff does not justify why a full week of advertising costs would be covered when only a day was lost, suggesting a lack of proportionality in the mitigation expenses.

- **Seven Circuit Decision:**
  - The plaintiff dismisses the Seven Circuit decision as immaterial, but it may provide insight into how contractual obligations are interpreted in relation to system failures and subsequent actions.

- **Etymology of "Incur":**
  - The plaintiff's reliance on the etymology of "incur" is not a strong legal argument. The policy terms and conditions, not the Latin root of words, should determine coverage.

- **Lack of Precedent:**
  - The plaintiff acknowledges the absence of precedent in the business interruption context, which suggests that the court should be cautious in extending coverage to voluntary mitigation expenses without clear guidance.

- **Fact Finder's Role:**
  - The plaintiff's argument that all issues should go to the fact finder overlooks the possibility that some issues may be decided as a matter of law if the policy language is clear.

- **Policy Interpretation:**
  - The plaintiff's interpretation of the policy may be overly broad, potentially leading to coverage for actions that extend beyond the intent of the policy and the reasonable expectations of the insurer.

In [71]:
system_failure = chain.invoke("Generate a detailed and comprehensive summary of the 'system failure' being argued. Include facts and quotes as needed to.")
Markdown(f"{system_failure}")

The discussion revolves around a 'system failure' experienced by Southwest Airlines and the subsequent insurance coverage dispute. The system failure led to significant disruptions, including the stranding of approximately 400,000 passengers. Southwest made a series of decisions to mitigate the impact on customers, which included offering refunds, vouchers, and other gestures of goodwill. These decisions were not legally mandated but were made unilaterally by Southwest as a business judgment to maintain customer relations and goodwill.

The crux of the argument is whether these voluntary, goodwill measures taken by Southwest in response to the system failure are covered under their business interruption insurance policy. The policy specifies coverage for losses "solely as a result of the system failure," which has become a point of contention. Southwest argues that all the customer compensation decisions were undertaken solely because of the system failure, suggesting that these should be covered by the insurance.

The insurer, on the other hand, contends that the measures go beyond what is covered by the policy. They argue that the policy is intended to put the insured in the position they would have been in if the system failure had not occurred, and does not cover the downstream effects such as PR or customer goodwill measures. The insurer maintains that the policy does not cover costs that are not a direct necessity or that go beyond restoring the company to the position it was in before the system failure.

The discussion also touches on the nature of the benefits provided to customers, including the 'Fair Saver' promo codes, which were linked to individual booking numbers and could cover up to eight passengers. The insurer challenges the nexus between these benefits and the system failure, particularly for passengers who did not travel during the disruption. Southwest defends this by stating that the actual usage of the benefits resulted in an average of 1.5 tickets per promo code, and that these measures were consistent with their historical practices in similar situations.

The debate also includes a legal analysis of what constitutes a loss that is "incurred" under the policy, with Southwest's counsel pointing to the etymology of the word "incur" to argue that it implies a self-inflicted negative consequence, which should be covered by the policy.

The court is tasked with determining whether the voluntary measures taken by Southwest are indeed covered losses under the insurance policy, considering the policy's language and the intent behind the business interruption coverage. The insurer suggests that the court's role is to differentiate between out-of-pocket costs directly associated with the system failure and benefits that may extend beyond that scope. The outcome of this dispute will hinge on the interpretation of the policy language and the factual circumstances surrounding the system failure and Southwest's response.